# Final Project Notebook

The report for this final project can be found at this [link](https://cybertraining-dsc.github.io/report/fa20-523-301/project/project/).

## Part 1 Importing the functions

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab

import pandas as pd
import io
import requests

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Setting up download function. This script will check to see if the filename has any data, and if it doesn't it will download from the specified url.

In [15]:
def download(url, filename):
    if len(filename.columns) >0:
        n=0
        #break
    else: 
        #url="https://github.com/cs109/2014_data/blob/master/countries.csv"
        
        s=requests.get(url).content
        filename=pd.read_csv(url, error_bad_lines=False)
        
    return filename

In [18]:
nBA_injuries = pd.DataFrame()
nBA_games = pd.DataFrame()
nBA_stats = pd.DataFrame()

In [19]:
print(nBA_injuries)

Empty DataFrame
Columns: []
Index: []


In [20]:
url='https://www.kaggle.com/ghopkins/nba-injuries-2010-2018?select=injuries_2010-2020.csv'
filename = nBA_injuries
nBA_injuries = download(url, filename)

url = 'https://www.kaggle.com/nathanlauga/nba-games?select=players.csv'
filename = nBA_games
nBA_games = download(url, filename)

url = 'https://www.kaggle.com/pablote/nba-enhanced-stats?select=2012-18_playerBoxScore.csv'
filename = nBA_stats
nBA_stats = download(url, filename)



b'Skipping line 6: expected 1 fields, saw 2\nSkipping line 9: expected 1 fields, saw 3\nSkipping line 10: expected 1 fields, saw 4\nSkipping line 21: expected 1 fields, saw 2\nSkipping line 33: expected 1 fields, saw 8\nSkipping line 42: expected 1 fields, saw 9\nSkipping line 46: expected 1 fields, saw 2\nSkipping line 47: expected 1 fields, saw 6\nSkipping line 52: expected 1 fields, saw 2\nSkipping line 53: expected 1 fields, saw 3\nSkipping line 54: expected 1 fields, saw 2\nSkipping line 55: expected 1 fields, saw 2\nSkipping line 56: expected 1 fields, saw 2\nSkipping line 78: expected 1 fields, saw 49\nSkipping line 88: expected 1 fields, saw 2\nSkipping line 89: expected 1 fields, saw 2\nSkipping line 90: expected 1 fields, saw 2\nSkipping line 91: expected 1 fields, saw 2\nSkipping line 98: expected 1 fields, saw 2\nSkipping line 100: expected 1 fields, saw 2\nSkipping line 108: expected 1 fields, saw 2\nSkipping line 169: expected 1 fields, saw 2\nSkipping line 170: expected 

In [17]:
print(nBA_injuries)

                                       <!DOCTYPE html>
0                                     <html lang="en">
1                                               <head>
2        <title>NBA Injuries from 2010-2020 | Kaggl...
3                             <meta charset="utf-8" />
4        <meta name="description" content="Analyze ...
..                                                 ...
138  <script nonce="h0ZzF4xxNwi4aDR7lQdPaw==" type=...
139                                             </div>
140                                            </main>
141                                            </body>
142                                            </html>

[143 rows x 1 columns]


In [21]:
print(nBA_games)

                                       <!DOCTYPE html>
0                                     <html lang="en">
1                                               <head>
2               <title>NBA games data | Kaggle</title>
3                             <meta charset="utf-8" />
4        <meta name="description" content="Dataset ...
..                                                 ...
139  <script nonce="na/vE0WFqoVRt20WVnT6xg==" type=...
140                                             </div>
141                                            </main>
142                                            </body>
143                                            </html>

[144 rows x 1 columns]


In [22]:
print(nBA_stats)

                                       <!DOCTYPE html>
0                                     <html lang="en">
1                                               <head>
2        <title>NBA Enhanced Box Score and Standing...
3                             <meta charset="utf-8" />
4        <meta name="description" content="Box Scor...
..                                                 ...
139  <script nonce="5VgMeNDg8vMUNzcyTL53vg==" type=...
140                                             </div>
141                                            </main>
142                                            </body>
143                                            </html>

[144 rows x 1 columns]


# Part 2 Building the Keras Model

A link for Keras for us to use can be found [here](https://keras.io/guides/sequential_model/).

In [2]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

Insert Layers

In [3]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

# Part 3 Conclusions

This is where the conclusions section will be typed